# Basic RNN
- Objective: to understand basics of RNN & LSTM

## Recurrent Neural Networks
- Feedforward neural networks (e.g. MLPs and CNNs) are powerful, but they are not optimized to handle "sequential" data
- In other words, they do not possess "memory" of previous inputs
- For instance, consider the case of translating a corpus. You need to consider the **"context"** to guess the next word to come forward

<img src="http://2.bp.blogspot.com/-9GIdV292xV4/UwOIy6B6koI/AAAAAAAAHi4/X6UGlyHI-_U/s1600/tumblr_ms5qzpFY671r9nm7io1_500.gif" style="width: 500px"/>

<br>
- RNNs are suitable for dealing with sequential format data since they have **"recurrent"** structure
- To put it differently, they keep the **"memory"** of earlier inputs in the sequence
</br>
<img src="http://www.wildml.com/wp-content/uploads/2015/09/rnn.jpg" style="width: 600px"/>

<br>
- However, in order to reduce the number of parameters, every layer of different time steps shares same parameters
</br>

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png" style="width: 600px"/>

## Load Dataset

In [1]:
import numpy as np

from sklearn.metrics import accuracy_score
from keras.datasets import reuters
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
# parameters for data load
num_words = 30000
maxlen = 50
test_split = 0.3

In [3]:
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words = num_words, maxlen = maxlen, test_split = test_split)

2113536/2110848 [==============================] - 1s 1us/step


In [4]:
# pad the sequences with zeros 
# padding parameter is set to 'post' => 0's are appended to end of sequences
X_train = pad_sequences(X_train, padding = 'post')
X_test = pad_sequences(X_test, padding = 'post')

In [5]:
X_train = np.array(X_train).reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1], 1))

In [6]:
y_data = np.concatenate((y_train, y_test))
y_data = to_categorical(y_data)

In [7]:
y_train = y_data[:1395]
y_test = y_data[1395:]

In [10]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1395, 49, 1)
(599, 49, 1)
(1395, 46)
(599, 46)


## 1. Vanilla RNN
- Vanilla RNNs have a simple structure
- However, they suffer from the problem of "long-term dependencies"
- Hence, they are not able to keep the **sequential memory" for long

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-SimpleRNN.png" style="width: 600px"/>

In [11]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Activation
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier

In [12]:
def vanilla_rnn():
    model = Sequential()
    model.add(SimpleRNN(50, input_shape = (49,1), return_sequences = False))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [13]:
model = KerasClassifier(build_fn = vanilla_rnn, epochs = 200, batch_size = 50, verbose = 1)

In [14]:
model.fit(X_train, y_train)

Epoch 1/200
1395/1395 [==============================] - 1s 617us/step - loss: 2.8998 - accuracy: 0.4272
Epoch 2/200
1395/1395 [==============================] - 0s 252us/step - loss: 1.4049 - accuracy: 0.6738
Epoch 3/200
1395/1395 [==============================] - 0s 242us/step - loss: 1.2218 - accuracy: 0.6968
Epoch 4/200
1395/1395 [==============================] - 0s 234us/step - loss: 1.1840 - accuracy: 0.7147
Epoch 5/200
1395/1395 [==============================] - 0s 275us/step - loss: 1.1695 - accuracy: 0.7147
Epoch 6/200
1395/1395 [==============================] - 0s 275us/step - loss: 1.1626 - accuracy: 0.7147
Epoch 7/200
1395/1395 [==============================] - 0s 269us/step - loss: 1.1572 - accuracy: 0.7147
Epoch 8/200
1395/1395 [==============================] - 0s 263us/step - loss: 1.1536 - accuracy: 0.7147
Epoch 9/200
1395/1395 [==============================] - 0s 231us/step - loss: 1.1524 - accuracy: 0.7147
Epoch 10/200
1395/1395 [==============================]

In [15]:
y_pred = model.predict(X_test)

599/599 [==============================] - 0s 206us/step


In [16]:
y_test_ = np.argmax(y_test, axis = 1)

In [17]:
print(accuracy_score(y_pred, y_test_))

0.7378964941569283


## 2. Stacked Vanilla RNN
- RNN layers can be stacked to form a deeper network

<img src="https://lh6.googleusercontent.com/rC1DSgjlmobtRxMPFi14hkMdDqSkEkuOX7EW_QrLFSymjasIM95Za2Wf-VwSC1Tq1sjJlOPLJ92q7PTKJh2hjBoXQawM6MQC27east67GFDklTalljlt0cFLZnPMdhp8erzO" style="width: 500px"/>

In [21]:
def stacked_vanilla_rnn():
    model = Sequential()
    model.add(SimpleRNN(50, input_shape = (49,1), return_sequences = True))   # return_sequences parameter has to be set True to stack
    model.add(SimpleRNN(50, return_sequences = False))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [22]:
model = KerasClassifier(build_fn = stacked_vanilla_rnn, epochs = 200, batch_size = 50, verbose = 1)

In [23]:
model.fit(X_train, y_train)

Epoch 1/200
1395/1395 [==============================] - 1s 841us/step - loss: 2.8363 - accuracy: 0.4717
Epoch 2/200
1395/1395 [==============================] - 1s 540us/step - loss: 1.2900 - accuracy: 0.7147
Epoch 3/200
1395/1395 [==============================] - 1s 537us/step - loss: 1.1703 - accuracy: 0.7147
Epoch 4/200
1395/1395 [==============================] - 1s 541us/step - loss: 1.1427 - accuracy: 0.7147
Epoch 5/200
1395/1395 [==============================] - 1s 535us/step - loss: 1.1189 - accuracy: 0.7147
Epoch 6/200
1395/1395 [==============================] - 1s 505us/step - loss: 1.0972 - accuracy: 0.7147
Epoch 7/200
1395/1395 [==============================] - 1s 549us/step - loss: 1.0873 - accuracy: 0.7147
Epoch 8/200
1395/1395 [==============================] - 1s 525us/step - loss: 1.0821 - accuracy: 0.7147
Epoch 9/200
1395/1395 [==============================] - 1s 539us/step - loss: 1.0744 - accuracy: 0.7147
Epoch 10/200
1395/1395 [==============================]

In [77]:
y_pred = model.predict(X_test)

500/599 [========================>.....] - ETA: 0s

In [78]:
print(accuracy_score(y_pred, y_test_))

0.746243739566


## 3. LSTM
- LSTM (long short-term memory) is an improved structure to solve the problem of long-term dependencies

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png" style="width: 600px"/>

In [79]:
from keras.layers import LSTM

In [80]:
def lstm():
    model = Sequential()
    model.add(LSTM(50, input_shape = (49,1), return_sequences = False))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [81]:
model = KerasClassifier(build_fn = lstm, epochs = 200, batch_size = 50, verbose = 1)

In [ ]:
model.fit(X_train, y_train)

In [83]:
y_pred = model.predict(X_test)

500/599 [========================>.....] - ETA: 0s

In [84]:
# accuracy improves by adopting LSTM structure
print(accuracy_score(y_pred, y_test_))

0.844741235392


## 4. Stacked LSTM
- LSTM layers can be stacked as well

In [85]:
def stacked_lstm():
    model = Sequential()
    model.add(LSTM(50, input_shape = (49,1), return_sequences = True))
    model.add(LSTM(50, return_sequences = False))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [86]:
model = KerasClassifier(build_fn = stacked_lstm, epochs = 200, batch_size = 50, verbose = 1)

In [ ]:
model.fit(X_train, y_train)

In [88]:
y_pred = model.predict(X_test)

500/599 [========================>.....] - ETA: 0s

In [89]:
print(accuracy_score(y_pred, y_test_))

0.858096828047
